In [3]:
import os
x = os.listdir()

In [ ]:
Atiku = []
for data in x:
        Atiku.append(data)
print("success")

success


In [1]:
import pandas as pd
for data in Atiku:
  print("processing " + data + "...")
  df = pd.read_json(data, lines=True)
df.head()

'import pandas as pd\nfor data in Atiku:\n  print("processing " + data + "...")\n  df = pd.read_json(data, lines=True)\ndf.head()'

In [ ]:
name, tweet, time, location, followers, friends, status_count = [],[],[],[],[],[],[]
#Get screen name
for line in df.user:
  n = line.get("screen_name")
  name.append(n)
#Get tweet while removing all non-characters
for line in df.text:
  t = str(line).lower()
  t.replace('[^a-zA-Z -]','')
  tweet.append(t)
#Get time
for line in df.created_at:
  time.append(line)
#Get location while converting it to lower case
for line in df.user:
  l = line.get('location')
  l = str(l).lower()
  location.append(l)
#Get number of followers
for line in df.user:
  f = line.get('followers_count')
  followers.append(f)
#Get friends
for line in  df.user:
  fr = line.get('friends_count')
  friends.append(fr)
for line in df.user:
  st = line.get('statuses_count')
  status_count.append(st)
print('done')

done


In [ ]:
Tweets = pd.DataFrame(columns = ('name', 'tweet', 'time', 'location', 'followers', 'friends', 'status_count' ))
Tweets['name'] = name
Tweets['tweet'] = tweet
Tweets['time'] = time
Tweets['location'] = location
Tweets['followers'] = followers
Tweets['friends'] = friends
Tweets['status_count'] = status_count

#Write tweets to csv
Tweets.to_csv('AtikuData.csv')

#Summarize the the data by some parameters
location_summary = Tweets.groupby('location').count().sort_values(by='name', ascending=False)
name_summary = Tweets.groupby('name').count().sort_values(by='tweet',ascending = False)
time_summary = Tweets.groupby('time').count().sort_values(by='name',ascending=False)
#Write the data to csv
location_summary.to_csv('location_summary.csv')
name_summary.to_csv('name_summary.csv')
time_summary.to_csv('time_summary.csv')

print('completed succesfully')

completed succesfully


In [ ]:
#To set pandas to view the entire width of each column
pd.set_option('display.max_colwidth',-1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [4]:
import pandas as pd
df = pd.read_csv('AtikuData.csv')
df.head()

,Unnamed: 0,name,tweet,time,location,followers,friends,status_count
0,0,Vendetta111,"@saharareporters @atiku just out of curiosity,...",2019-01-03 14:09:53+00:00,malaysia,64,230,188
1,1,EddyJay85897135,@ogundamisi @officialpdpnig @atiku @awtambuwal...,2019-01-03 14:09:53+00:00,commonwealth,18,51,377
2,2,abradbassam,@youthsforatiku where will the money come from...,2019-01-03 14:09:55+00:00,none,7,48,441
3,3,doncharlo_1,@kcnaija fear they catch them. i honestly exp...,2019-01-03 14:09:59+00:00,none,64,378,1204
4,4,MuhdSheikh1,"rt @fkeyamo: throw-in the towel, keyamo hits a...",2019-01-03 14:10:00+00:00,nigeria,436,1169,21362


In [6]:
#Remove Unnamed: 0 column
df.drop('Unnamed: 0',axis=1, inplace= True)

#Create new columns to separate time from date 
df['date'] = df.loc[:,'time'].map(lambda x: x.replace(x[10:],''))
df['time'] = df.loc[:,'time'].map(lambda x: x.replace(x[:10],''))

#sort the tweets by their date and time
Tweets_by_Date = df.sort_values(['time','date'],ascending = True)

Tweets_by_Date.head()

,name,tweet,time,location,followers,friends,status_count,date
0,Vendetta111,"@saharareporters @atiku just out of curiosity,...",14:09:53+00:00,malaysia,64,230,188,2019-01-03
1,EddyJay85897135,@ogundamisi @officialpdpnig @atiku @awtambuwal...,14:09:53+00:00,commonwealth,18,51,377,2019-01-03
2,abradbassam,@youthsforatiku where will the money come from...,14:09:55+00:00,none,7,48,441,2019-01-03
3,doncharlo_1,@kcnaija fear they catch them. i honestly exp...,14:09:59+00:00,none,64,378,1204,2019-01-03
4,MuhdSheikh1,"rt @fkeyamo: throw-in the towel, keyamo hits a...",14:10:00+00:00,nigeria,436,1169,21362,2019-01-03


In [7]:
#Sentiment Analysis
Candidates_Tweets = Tweets_by_Date.tweet
#Change to lower case
Candidates_Tweets = Candidates_Tweets.map(lambda x: str(x).lower())
#Remove non-characters
Candidates_Tweets = Candidates_Tweets.map(lambda x: x.replace('[^a-zA-Z]', ''))
#Candidates_Tweets.head()
#Define the variables
sentiment = []
polarity = 0
positive = 0
negative = 0
neutral = 0

from textblob import TextBlob
import matplotlib.pyplot as plt
for tweet in Candidates_Tweets:
  analysis = TextBlob(tweet)
  polarity += analysis.sentiment.polarity
  if analysis.sentiment.polarity ==0:
    sentiment.append('neutral')
  elif analysis.sentiment.polarity < 0.00:
    sentiment.append('negative')
  elif analysis.sentiment.polarity > 0.00:
    sentiment.append('positve')
print('done with sentiment analysis')

#Add sentiments as a column to Tweets_by_Date
Tweets_by_Date['sentiments'] = sentiment
Tweets_by_Date.head() 

done with sentiment analysis


,name,tweet,time,location,followers,friends,status_count,date,sentiments
0,Vendetta111,"@saharareporters @atiku just out of curiosity,...",14:09:53+00:00,malaysia,64,230,188,2019-01-03,neutral
1,EddyJay85897135,@ogundamisi @officialpdpnig @atiku @awtambuwal...,14:09:53+00:00,commonwealth,18,51,377,2019-01-03,neutral
2,abradbassam,@youthsforatiku where will the money come from...,14:09:55+00:00,none,7,48,441,2019-01-03,positve
3,doncharlo_1,@kcnaija fear they catch them. i honestly exp...,14:09:59+00:00,none,64,378,1204,2019-01-03,positve
4,MuhdSheikh1,"rt @fkeyamo: throw-in the towel, keyamo hits a...",14:10:00+00:00,nigeria,436,1169,21362,2019-01-03,neutral


In [9]:
#Save Tweets by date as a csv file, then analyse with Power BI, Tablaeu or any other convenient visualization tool
Tweets_by_Date.to_csv('Tweets_by_Date.csv')